# **Assignment Day 13 (Mochamad Ananda Radya Mikola)**

Task Assigment Day 13 - Big Data Analytics with PySpark Part 2

## **1. Data Preparation (Extract)**

Mempersipakan data dari file CSV yang diberikan oleh Mas Albert melalui link drive.


In [1]:
#Menginstal PySpark

! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=55e20d744f36ab63ed1a1c7a56358602ab0cc9183e18f957bd1275bbac3d8356
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
#Menginstal kaleido untuk ngeload atau export fig dari hasil analisis nanti menjadi PNG

! pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.8 MB/s eta 0:00:00


In [3]:
#Melakukan import library yang digunakan sepanjang proses cleaning,transform dan load serta analisis nanti

from google.colab import drive
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, dayofweek, expr, year, split, explode, corr
import plotly.express as px

In [4]:
#Melakukan mounting ke drive saya, dimana saya menaruh filenya yang diperlukan di drive saya

drive.mount('/content/drive')

#Melakukan path directory ke file-file yang akan digunakan nanti dan dilakukan read menggunakan SparkSession

file_path_attendance = '/content/drive/MyDrive/Day 13/Dataset/course_attendance.csv'
file_path_course = '/content/drive/MyDrive/Day 13/Dataset/course.csv'
file_path_enroll = '/content/drive/MyDrive/Day 13/Dataset/enrollment.csv'
file_path_schedule = '/content/drive/MyDrive/Day 13/Dataset/schedule.csv'

Mounted at /content/drive


In [5]:
ss = SparkSession.builder.appName('AssignmentDay13').master('local').getOrCreate()

In [6]:
df_course_attendance = ss.read.csv (file_path_attendance, header=True, inferSchema=True)

In [7]:
df_schedule = ss.read.csv (file_path_schedule, header=True, inferSchema=True)

In [8]:
df_course = ss.read.csv (file_path_course, header=True, inferSchema=True)

In [9]:
df_enrollment = ss.read.csv (file_path_enroll, header=True, inferSchema=True)

## **2. Data Cleaning and Transform**

Melakukan pembersihan data dan mengubah format-format data yang belum sesuai menjadi format yang sesuai dengan data sebenarnya.

### **Data Course Attendace**

In [10]:
df_course_attendance.show(10)
df_course_attendance.printSchema()

+---+----------+-----------+---------+
| ID|STUDENT_ID|SCHEDULE_ID|ATTEND_DT|
+---+----------+-----------+---------+
|  1|         1|          2|10-Sep-19|
|  2|         1|          2|11-Sep-19|
|  3|         1|          2|17-Sep-19|
|  4|         1|          2|18-Sep-19|
|  5|         1|          2|24-Sep-19|
|  6|         1|          2|25-Sep-19|
|  7|         1|          2|01-Oct-19|
|  8|         1|          2|02-Oct-19|
|  9|         1|          2|08-Oct-19|
| 10|         1|          2|09-Oct-19|
+---+----------+-----------+---------+
only showing top 10 rows

root
 |-- ID: integer (nullable = true)
 |-- STUDENT_ID: integer (nullable = true)
 |-- SCHEDULE_ID: integer (nullable = true)
 |-- ATTEND_DT: string (nullable = true)



In [11]:
#Saya mengubah nama kolomnya agar lebih rapih dan mudah saat melakukan sql join nanti

df_course_attendance = df_course_attendance \
    .withColumnRenamed("ID", "ca_id") \
    .withColumnRenamed("STUDENT_ID", "student_id") \
    .withColumnRenamed("SCHEDULE_ID", "schedule_id") \
    .withColumnRenamed("ATTEND_DT", "attend_date")
df_course_attendance.show(10)

+-----+----------+-----------+-----------+
|ca_id|student_id|schedule_id|attend_date|
+-----+----------+-----------+-----------+
|    1|         1|          2|  10-Sep-19|
|    2|         1|          2|  11-Sep-19|
|    3|         1|          2|  17-Sep-19|
|    4|         1|          2|  18-Sep-19|
|    5|         1|          2|  24-Sep-19|
|    6|         1|          2|  25-Sep-19|
|    7|         1|          2|  01-Oct-19|
|    8|         1|          2|  02-Oct-19|
|    9|         1|          2|  08-Oct-19|
|   10|         1|          2|  09-Oct-19|
+-----+----------+-----------+-----------+
only showing top 10 rows



In [12]:
#saya buat format attend_date nya menjadi format date yang benar

df_course_attendance = df_course_attendance \
    .withColumn('attend_date',F.to_date('attend_date', format='dd-MMM-yy'))
df_course_attendance.show(10)
df_course_attendance.printSchema()

+-----+----------+-----------+-----------+
|ca_id|student_id|schedule_id|attend_date|
+-----+----------+-----------+-----------+
|    1|         1|          2| 2019-09-10|
|    2|         1|          2| 2019-09-11|
|    3|         1|          2| 2019-09-17|
|    4|         1|          2| 2019-09-18|
|    5|         1|          2| 2019-09-24|
|    6|         1|          2| 2019-09-25|
|    7|         1|          2| 2019-10-01|
|    8|         1|          2| 2019-10-02|
|    9|         1|          2| 2019-10-08|
|   10|         1|          2| 2019-10-09|
+-----+----------+-----------+-----------+
only showing top 10 rows

root
 |-- ca_id: integer (nullable = true)
 |-- student_id: integer (nullable = true)
 |-- schedule_id: integer (nullable = true)
 |-- attend_date: date (nullable = true)



### **Data Course**

In [13]:
df_course.show()
df_course.printSchema()

+---+--------------------+
| ID|                NAME|
+---+--------------------+
|  1|Introduction to A...|
|  2|Natural Language ...|
|  3|         Data Mining|
|  4|Applied Data Science|
|  5|           Statistic|
|  6|Introduction to O...|
|  7|  Python Programming|
|  8|Information Retri...|
|  9|          Blockchain|
| 10|  Internet of Things|
+---+--------------------+

root
 |-- ID: integer (nullable = true)
 |-- NAME: string (nullable = true)



In [14]:
#Saya mengubah nama kolomnya agar lebih rapih dan mudah saat melakukan sql join nanti

df_course = df_course \
    .withColumnRenamed("ID", "course_id") \
    .withColumnRenamed("NAME", "course_name")
df_course.show()

+---------+--------------------+
|course_id|         course_name|
+---------+--------------------+
|        1|Introduction to A...|
|        2|Natural Language ...|
|        3|         Data Mining|
|        4|Applied Data Science|
|        5|           Statistic|
|        6|Introduction to O...|
|        7|  Python Programming|
|        8|Information Retri...|
|        9|          Blockchain|
|       10|  Internet of Things|
+---------+--------------------+



### **Data Enrollment**

In [15]:
df_enrollment.show(10)
df_enrollment.printSchema()

+---+----------+-----------+-------------+--------+---------+
| ID|STUDENT_ID|SCHEDULE_ID|ACADEMIC_YEAR|SEMESTER|ENROLL_DT|
+---+----------+-----------+-------------+--------+---------+
|  1|         1|          2|    2019/2020|       1|17-Aug-19|
|  2|         1|          4|    2019/2020|       1|17-Aug-19|
|  3|         2|          1|    2019/2020|       1|17-Aug-19|
|  4|         2|          2|    2019/2020|       1|17-Aug-19|
|  5|         2|          3|    2019/2020|       1|17-Aug-19|
|  6|         6|          1|    2019/2020|       1|17-Aug-19|
|  7|         6|          2|    2019/2020|       1|17-Aug-19|
|  8|         7|          3|    2019/2020|       1|17-Aug-19|
|  9|        10|          4|    2019/2020|       1|17-Aug-19|
| 10|         1|          7|    2019/2020|       2|17-Aug-19|
+---+----------+-----------+-------------+--------+---------+
only showing top 10 rows

root
 |-- ID: integer (nullable = true)
 |-- STUDENT_ID: integer (nullable = true)
 |-- SCHEDULE_ID: integ

In [16]:
#Saya mengubah nama kolomnya agar lebih rapih dan mudah saat melakukan sql join nanti

df_enrollment = df_enrollment \
    .withColumnRenamed("ID", "enrollment_id") \
    .withColumnRenamed("STUDENT_ID", "student_id") \
    .withColumnRenamed("SCHEDULE_ID", "schedule_id") \
    .withColumnRenamed("ACADEMIC_YEAR", "academic_year") \
    .withColumnRenamed("SEMESTER", "semester") \
    .withColumnRenamed("ENROLL_DT", "enroll_date")

#saya buat format enroll_date nya menjadi format date yang benar

df_enrollment = df_enrollment.withColumn('enroll_date',F.to_date('enroll_date', format='dd-MMM-yy'))
df_enrollment.show(10)
df_enrollment.printSchema()

+-------------+----------+-----------+-------------+--------+-----------+
|enrollment_id|student_id|schedule_id|academic_year|semester|enroll_date|
+-------------+----------+-----------+-------------+--------+-----------+
|            1|         1|          2|    2019/2020|       1| 2019-08-17|
|            2|         1|          4|    2019/2020|       1| 2019-08-17|
|            3|         2|          1|    2019/2020|       1| 2019-08-17|
|            4|         2|          2|    2019/2020|       1| 2019-08-17|
|            5|         2|          3|    2019/2020|       1| 2019-08-17|
|            6|         6|          1|    2019/2020|       1| 2019-08-17|
|            7|         6|          2|    2019/2020|       1| 2019-08-17|
|            8|         7|          3|    2019/2020|       1| 2019-08-17|
|            9|        10|          4|    2019/2020|       1| 2019-08-17|
|           10|         1|          7|    2019/2020|       2| 2019-08-17|
+-------------+----------+-----------+

### **Data Schedule**

In [17]:
df_schedule.show()
df_schedule.printSchema()

+---+---------+-----------+---------+---------+-----------+
| ID|COURSE_ID|LECTURER_ID| START_DT|   END_DT|COURSE_DAYS|
+---+---------+-----------+---------+---------+-----------+
|  1|        1|         11|09-Sep-19|08-Dec-19|      2,4,6|
|  2|        3|          4|09-Sep-19|08-Dec-19|        3,4|
|  3|        4|         13|09-Sep-19|08-Dec-19|        2,5|
|  4|        5|         28|09-Sep-19|08-Dec-19|          3|
|  5|        2|         56|27-Jan-20|26-Apr-20|        2,4|
|  6|        7|          1|27-Jan-20|26-Apr-20|        4,5|
|  7|        1|         11|27-Jan-20|26-Apr-20|      2,4,6|
|  8|        7|         22|27-Jan-20|26-Apr-20|        2,3|
|  9|        8|         56|27-Jan-20|26-Apr-20|          5|
| 10|        3|         37|27-Jan-20|26-Apr-20|        5,6|
+---+---------+-----------+---------+---------+-----------+

root
 |-- ID: integer (nullable = true)
 |-- COURSE_ID: integer (nullable = true)
 |-- LECTURER_ID: integer (nullable = true)
 |-- START_DT: string (nullable =

In [18]:
#Saya mengubah nama kolomnya agar lebih rapih dan mudah saat melakukan sql join nanti

df_schedule = df_schedule \
    .withColumnRenamed("ID", "schedule_id") \
    .withColumnRenamed("COURSE_ID", "course_id") \
    .withColumnRenamed("LECTURER_ID", "lecturer_id") \
    .withColumnRenamed("START_DT", "start_date") \
    .withColumnRenamed("END_DT", "end_date") \
    .withColumnRenamed("COURSE_DAYS", "course_days")

#saya buat format end_date dan start_date nya menjadi format date yang benar

df_schedule = df_schedule \
    .withColumn('start_date', F.to_date('start_date', format='dd-MMM-yy')) \
    .withColumn('end_date', F.to_date('end_date', format='dd-MMM-yy'))

df_schedule.show()
df_schedule.printSchema()

+-----------+---------+-----------+----------+----------+-----------+
|schedule_id|course_id|lecturer_id|start_date|  end_date|course_days|
+-----------+---------+-----------+----------+----------+-----------+
|          1|        1|         11|2019-09-09|2019-12-08|      2,4,6|
|          2|        3|          4|2019-09-09|2019-12-08|        3,4|
|          3|        4|         13|2019-09-09|2019-12-08|        2,5|
|          4|        5|         28|2019-09-09|2019-12-08|          3|
|          5|        2|         56|2020-01-27|2020-04-26|        2,4|
|          6|        7|          1|2020-01-27|2020-04-26|        4,5|
|          7|        1|         11|2020-01-27|2020-04-26|      2,4,6|
|          8|        7|         22|2020-01-27|2020-04-26|        2,3|
|          9|        8|         56|2020-01-27|2020-04-26|          5|
|         10|        3|         37|2020-01-27|2020-04-26|        5,6|
+-----------+---------+-----------+----------+----------+-----------+

root
 |-- schedule_

In [19]:
#Saya membuat urutan hari berdasarkan angka dalam "days", kemudian mengubah data pada "course_day" menjadi nama hari yang sesuai

days = {
    "1": "Sunday",
    "2": "Monday",
    "3": "Tuesday",
    "4": "Wednesday",
    "5": "Thursday",
    "6": "Friday",
    "7": "Saturday"
}

for day, name in days.items():
  df_schedule = df_schedule.withColumn("course_days", expr(f"replace(course_days, '{day}', '{name}')"))

df_schedule.show()
df_schedule.printSchema()

+-----------+---------+-----------+----------+----------+--------------------+
|schedule_id|course_id|lecturer_id|start_date|  end_date|         course_days|
+-----------+---------+-----------+----------+----------+--------------------+
|          1|        1|         11|2019-09-09|2019-12-08|Monday,Wednesday,...|
|          2|        3|          4|2019-09-09|2019-12-08|   Tuesday,Wednesday|
|          3|        4|         13|2019-09-09|2019-12-08|     Monday,Thursday|
|          4|        5|         28|2019-09-09|2019-12-08|             Tuesday|
|          5|        2|         56|2020-01-27|2020-04-26|    Monday,Wednesday|
|          6|        7|          1|2020-01-27|2020-04-26|  Wednesday,Thursday|
|          7|        1|         11|2020-01-27|2020-04-26|Monday,Wednesday,...|
|          8|        7|         22|2020-01-27|2020-04-26|      Monday,Tuesday|
|          9|        8|         56|2020-01-27|2020-04-26|            Thursday|
|         10|        3|         37|2020-01-27|2020-0

## **3. Data Analysis and Visualization**

Melakukan analisis terhadap data yang telah dibersihkan dan dirapihkan formatnya, serta melakukan visualisasi dari data yang telah dianalisis.

### **Analisis 1**

Mengetahui berapa banyak siswa yang mengikuti pada setiap kursus.

Adapaun data yang dibutuhkan dengan melihat semua siswa yang melakukan enrollment ke kursus yang mereka inginkan, sehingga nantinya bisa terhitung mana kursus yang paling diminati dan mana kursus yang paling sedikit diminati.

In [20]:
df_course.show()
df_enrollment.show()
df_schedule.show()

+---------+--------------------+
|course_id|         course_name|
+---------+--------------------+
|        1|Introduction to A...|
|        2|Natural Language ...|
|        3|         Data Mining|
|        4|Applied Data Science|
|        5|           Statistic|
|        6|Introduction to O...|
|        7|  Python Programming|
|        8|Information Retri...|
|        9|          Blockchain|
|       10|  Internet of Things|
+---------+--------------------+

+-------------+----------+-----------+-------------+--------+-----------+
|enrollment_id|student_id|schedule_id|academic_year|semester|enroll_date|
+-------------+----------+-----------+-------------+--------+-----------+
|            1|         1|          2|    2019/2020|       1| 2019-08-17|
|            2|         1|          4|    2019/2020|       1| 2019-08-17|
|            3|         2|          1|    2019/2020|       1| 2019-08-17|
|            4|         2|          2|    2019/2020|       1| 2019-08-17|
|            5|    

In [21]:
#Menggabungkan tabel schedule, enrollment, dan course. Kemudian menggunakan select untuk memperlihatkan data yang lebih rapih

df_analys1 = df_schedule \
    .join(df_enrollment, (df_schedule['schedule_id'] == df_enrollment['schedule_id']), 'inner') \
    .join(df_course, (df_course['course_id'] == df_schedule['course_id']), 'inner') \
    .select(
            'course_name',
            'lecturer_id',
            'start_date',
            'start_date',
            'end_date',
            'course_days',
            'enrollment_id',
            'student_id',
            'academic_year',
            'semester',
            'enroll_date'
            )

df_analys1.show()

+--------------------+-----------+----------+----------+----------+--------------------+-------------+----------+-------------+--------+-----------+
|         course_name|lecturer_id|start_date|start_date|  end_date|         course_days|enrollment_id|student_id|academic_year|semester|enroll_date|
+--------------------+-----------+----------+----------+----------+--------------------+-------------+----------+-------------+--------+-----------+
|         Data Mining|          4|2019-09-09|2019-09-09|2019-12-08|   Tuesday,Wednesday|            1|         1|    2019/2020|       1| 2019-08-17|
|           Statistic|         28|2019-09-09|2019-09-09|2019-12-08|             Tuesday|            2|         1|    2019/2020|       1| 2019-08-17|
|Introduction to A...|         11|2019-09-09|2019-09-09|2019-12-08|Monday,Wednesday,...|            3|         2|    2019/2020|       1| 2019-08-17|
|         Data Mining|          4|2019-09-09|2019-09-09|2019-12-08|   Tuesday,Wednesday|            4|    

In [22]:
#Melakukan analisis dengan groupBy kolom "course_name" dan menghitung jumlah siswa pada setiap kursus

df_analys1_clean = df_analys1 \
    .groupBy('course_name') \
    .count() \
    .withColumnRenamed('count', 'total_student')
df_analys1_clean.show()

+--------------------+-------------+
|         course_name|total_student|
+--------------------+-------------+
|  Python Programming|            1|
|         Data Mining|            5|
|           Statistic|            2|
|Applied Data Science|            2|
|Introduction to A...|            4|
|Natural Language ...|            1|
+--------------------+-------------+



In [23]:
#Melakukan visualisasi dari data yang telah di analisis

pandas_df_analys1_clean = df_analys1_clean.toPandas()
fig1 = px.bar(pandas_df_analys1_clean, x='course_name', y='total_student', title= "Jumlah Siswa Pada Setiap Kursus")
fig1.show()

### **Analisis 2**

Mengetahui total kehadiran siswa pada setiap kursus di tahun 2019 dan 2020.

Adapun data yang dibutuhkan adalah data absensi siswa pada kursus yang telah mereka pilih. Sehingga nantinya dapat ditentukan kursus mana yang memiliki tingkat kehadiran siswa yang tinggi dan mana yang memiliki tingkat kehadiran yang rendah pada tahun 2019 dan 2020.

In [24]:
df_course_attendance.show(5)
df_course.show()
df_schedule.show()

+-----+----------+-----------+-----------+
|ca_id|student_id|schedule_id|attend_date|
+-----+----------+-----------+-----------+
|    1|         1|          2| 2019-09-10|
|    2|         1|          2| 2019-09-11|
|    3|         1|          2| 2019-09-17|
|    4|         1|          2| 2019-09-18|
|    5|         1|          2| 2019-09-24|
+-----+----------+-----------+-----------+
only showing top 5 rows

+---------+--------------------+
|course_id|         course_name|
+---------+--------------------+
|        1|Introduction to A...|
|        2|Natural Language ...|
|        3|         Data Mining|
|        4|Applied Data Science|
|        5|           Statistic|
|        6|Introduction to O...|
|        7|  Python Programming|
|        8|Information Retri...|
|        9|          Blockchain|
|       10|  Internet of Things|
+---------+--------------------+

+-----------+---------+-----------+----------+----------+--------------------+
|schedule_id|course_id|lecturer_id|start_date|

In [25]:
#Menggabungkan tabel course_attendance, schedule, dan course. Kemudian menggunakan select untuk memperlihatkan data yang lebih rapih

df_analys2 = df_course_attendance \
    .join(df_schedule, (df_course_attendance['schedule_id'] == df_schedule['schedule_id']), 'inner') \
    .join(df_course, (df_course['course_id'] == df_schedule['course_id']), 'inner') \
    .select(
        'ca_id',
        'student_id',
        'course_name',
        'course_days',
        'attend_date',
        'start_date',
        'end_date'
    )

df_analys2.show(10)

+-----+----------+-----------+-----------------+-----------+----------+----------+
|ca_id|student_id|course_name|      course_days|attend_date|start_date|  end_date|
+-----+----------+-----------+-----------------+-----------+----------+----------+
|    1|         1|Data Mining|Tuesday,Wednesday| 2019-09-10|2019-09-09|2019-12-08|
|    2|         1|Data Mining|Tuesday,Wednesday| 2019-09-11|2019-09-09|2019-12-08|
|    3|         1|Data Mining|Tuesday,Wednesday| 2019-09-17|2019-09-09|2019-12-08|
|    4|         1|Data Mining|Tuesday,Wednesday| 2019-09-18|2019-09-09|2019-12-08|
|    5|         1|Data Mining|Tuesday,Wednesday| 2019-09-24|2019-09-09|2019-12-08|
|    6|         1|Data Mining|Tuesday,Wednesday| 2019-09-25|2019-09-09|2019-12-08|
|    7|         1|Data Mining|Tuesday,Wednesday| 2019-10-01|2019-09-09|2019-12-08|
|    8|         1|Data Mining|Tuesday,Wednesday| 2019-10-02|2019-09-09|2019-12-08|
|    9|         1|Data Mining|Tuesday,Wednesday| 2019-10-08|2019-09-09|2019-12-08|
|   

In [26]:
#Melakukan filtering dan analisis berdasarkan data sesuai dengan kondisi yang dinginkan, yaitu data pada tahun 2019

df_analys2_filtered_2019 = df_analys2 \
    .filter(year(col('attend_date')) == 2019) \
    .groupBy('course_name').count().withColumnRenamed('count', 'total_attend')

df_analys2_filtered_2019.show(10)

+--------------------+------------+
|         course_name|total_attend|
+--------------------+------------+
|         Data Mining|          78|
|           Statistic|          19|
|Applied Data Science|          44|
|Introduction to A...|          66|
+--------------------+------------+



In [27]:
#Melakukan visualisasi dari data yang telah di analisis khususnya data tahun 2019

pandas_df_analys2_filtered_2019 = df_analys2_filtered_2019.toPandas()
fig2a = px.bar(pandas_df_analys2_filtered_2019, x='course_name', y='total_attend', title= "Total Kehadiran Siswa Tahun 2019")
fig2a.show()

In [28]:
#Melakukan filtering dan analisis berdasarkan data sesuai dengan kondisi yang dinginkan, yaitu data pada tahun 2020

df_analys2_filtered_2020 = df_analys2 \
    .filter(year(col('attend_date')) == 2020) \
    .groupBy('course_name').count().withColumnRenamed('count', 'total_attend')

df_analys2_filtered_2020.show(10)

+--------------------+------------+
|         course_name|total_attend|
+--------------------+------------+
|  Python Programming|          19|
|         Data Mining|          50|
|Introduction to A...|          78|
|Natural Language ...|          26|
+--------------------+------------+



In [29]:
#Melakukan visualisasi dari data yang telah di analisis khususnya data tahun 2020

pandas_df_analys2_filtered_2020 = df_analys2_filtered_2020.toPandas()
fig2b = px.bar(pandas_df_analys2_filtered_2020, x='course_name', y='total_attend', title= "Total Kehadiran Siswa Tahun 2020")
fig2b.show()

### **Analisis 3**

Mengetahui pola hari dalam seminggu di mana kursus yang paling sering dijadwalkan pada tahun 2019 dan 2020.

Adapun data yang dibutuhkan adalah data schedule kursus. Sehingga nantinya dapat memperlihatkan di mana hari yang paling banyak kursus berjalan di setiap  tahun 2019 dan 2020.


In [30]:
df_schedule.show()
df_course.show()

+-----------+---------+-----------+----------+----------+--------------------+
|schedule_id|course_id|lecturer_id|start_date|  end_date|         course_days|
+-----------+---------+-----------+----------+----------+--------------------+
|          1|        1|         11|2019-09-09|2019-12-08|Monday,Wednesday,...|
|          2|        3|          4|2019-09-09|2019-12-08|   Tuesday,Wednesday|
|          3|        4|         13|2019-09-09|2019-12-08|     Monday,Thursday|
|          4|        5|         28|2019-09-09|2019-12-08|             Tuesday|
|          5|        2|         56|2020-01-27|2020-04-26|    Monday,Wednesday|
|          6|        7|          1|2020-01-27|2020-04-26|  Wednesday,Thursday|
|          7|        1|         11|2020-01-27|2020-04-26|Monday,Wednesday,...|
|          8|        7|         22|2020-01-27|2020-04-26|      Monday,Tuesday|
|          9|        8|         56|2020-01-27|2020-04-26|            Thursday|
|         10|        3|         37|2020-01-27|2020-0

In [31]:
#Menggabungkan tabel schedule dan course. Kemudian menggunakan select untuk memperlihatkan data yang lebih rapih

df_analys3 = df_schedule \
    .join(df_course, (df_schedule['course_id'] == df_course['course_id']), 'inner') \
    .select(
        'schedule_id',
        'lecturer_id',
        'course_name',
        'course_days',
        'start_date',
        'end_date'
    )

df_analys3.show()

+-----------+-----------+--------------------+--------------------+----------+----------+
|schedule_id|lecturer_id|         course_name|         course_days|start_date|  end_date|
+-----------+-----------+--------------------+--------------------+----------+----------+
|          7|         11|Introduction to A...|Monday,Wednesday,...|2020-01-27|2020-04-26|
|          1|         11|Introduction to A...|Monday,Wednesday,...|2019-09-09|2019-12-08|
|          5|         56|Natural Language ...|    Monday,Wednesday|2020-01-27|2020-04-26|
|         10|         37|         Data Mining|     Thursday,Friday|2020-01-27|2020-04-26|
|          2|          4|         Data Mining|   Tuesday,Wednesday|2019-09-09|2019-12-08|
|          3|         13|Applied Data Science|     Monday,Thursday|2019-09-09|2019-12-08|
|          4|         28|           Statistic|             Tuesday|2019-09-09|2019-12-08|
|          8|         22|  Python Programming|      Monday,Tuesday|2020-01-27|2020-04-26|
|         

In [32]:
#Melakukan filtering berdasarkan data sesuai dengan kondisi yang dinginkan, yaitu data pada tahun 2019

df_analys3_filtered_2019 = df_analys3.filter(year(col('start_date')) == 2019)
df_analys3_filtered_2019.show()

+-----------+-----------+--------------------+--------------------+----------+----------+
|schedule_id|lecturer_id|         course_name|         course_days|start_date|  end_date|
+-----------+-----------+--------------------+--------------------+----------+----------+
|          1|         11|Introduction to A...|Monday,Wednesday,...|2019-09-09|2019-12-08|
|          2|          4|         Data Mining|   Tuesday,Wednesday|2019-09-09|2019-12-08|
|          3|         13|Applied Data Science|     Monday,Thursday|2019-09-09|2019-12-08|
|          4|         28|           Statistic|             Tuesday|2019-09-09|2019-12-08|
+-----------+-----------+--------------------+--------------------+----------+----------+



In [33]:
#Melakukan analisis sesuai yang ingin dicari dengan berdasarkan data yang telah dilakukan filtering sebelumnya

df_analys3_2019_clean = df_analys3_filtered_2019 \
    .withColumn('days', split(col('course_days'), ',')) \
    .withColumn('day', explode(col('days'))) \
    .groupBy('day') \
    .count() \
    .withColumnRenamed('count', 'total_course') \
    .orderBy(
        expr
      (
        "CASE WHEN day = 'Monday' THEN 1 \
        WHEN day = 'Tuesday' THEN 2 \
        WHEN day = 'Wednesday' THEN 3 \
        WHEN day = 'Thursday' THEN 4 \
        WHEN day = 'Friday' THEN 5 \
        ELSE 6 END"
        )
    )

df_analys3_2019_clean.show()

+---------+------------+
|      day|total_course|
+---------+------------+
|   Monday|           2|
|  Tuesday|           2|
|Wednesday|           2|
| Thursday|           1|
|   Friday|           1|
+---------+------------+



In [34]:
#Melakukan visualisasi dari data yang telah di analisis khususnya data tahun 2019

pandas_df_analys3_2019_clean = df_analys3_2019_clean.toPandas()
fig3a = px.bar(pandas_df_analys3_2019_clean, x='day', y='total_course', title= "Total Kursus Aktif Dalam Seminggu Tahun 2019")
fig3a.show()

In [35]:
#Melakukan filtering berdasarkan data sesuai dengan kondisi yang dinginkan, yaitu data pada tahun 2020

df_analys3_filtered_2020 = df_analys3.filter(year(col('start_date')) == 2020)
df_analys3_filtered_2020.show()

+-----------+-----------+--------------------+--------------------+----------+----------+
|schedule_id|lecturer_id|         course_name|         course_days|start_date|  end_date|
+-----------+-----------+--------------------+--------------------+----------+----------+
|          7|         11|Introduction to A...|Monday,Wednesday,...|2020-01-27|2020-04-26|
|          5|         56|Natural Language ...|    Monday,Wednesday|2020-01-27|2020-04-26|
|         10|         37|         Data Mining|     Thursday,Friday|2020-01-27|2020-04-26|
|          8|         22|  Python Programming|      Monday,Tuesday|2020-01-27|2020-04-26|
|          6|          1|  Python Programming|  Wednesday,Thursday|2020-01-27|2020-04-26|
|          9|         56|Information Retri...|            Thursday|2020-01-27|2020-04-26|
+-----------+-----------+--------------------+--------------------+----------+----------+



In [36]:
#Melakukan analisis sesuai yang ingin dicari dengan berdasarkan data yang telah dilakukan filtering sebelumnya

df_analys3_2020_clean = df_analys3_filtered_2020 \
    .withColumn('days', split(col('course_days'), ',')) \
    .withColumn('day', explode(col('days'))) \
    .groupBy('day') \
    .count() \
    .withColumnRenamed('count', 'total_course') \
    .orderBy(
        expr
      (
        "CASE WHEN day = 'Monday' THEN 1 \
        WHEN day = 'Tuesday' THEN 2 \
        WHEN day = 'Wednesday' THEN 3 \
        WHEN day = 'Thursday' THEN 4 \
        WHEN day = 'Friday' THEN 5 \
        ELSE 6 END"
        )
    )

df_analys3_2020_clean.show()

+---------+------------+
|      day|total_course|
+---------+------------+
|   Monday|           3|
|  Tuesday|           1|
|Wednesday|           3|
| Thursday|           3|
|   Friday|           2|
+---------+------------+



In [37]:
#Melakukan visualisasi dari data yang telah di analisis khususnya data tahun 2020

pandas_df_analys3_2020_clean = df_analys3_2020_clean.toPandas()
fig3b = px.bar(pandas_df_analys3_2020_clean, x='day', y='total_course', title= "Total Kursus Aktif Dalam Seminggu Tahun 2020")
fig3b.show()

### **Analisis 4**

Mengetahui total hari mengajar kursus oleh setiap lectur pada tahun ajaran 2019/2020.

Adapun data yang dibutuhkan kurang lebih sama pada anlisis 3 yaitu data schedule yang terdapat data lecturer_id. Sehingga nantinya dapat memperlihatkan berapa jumlah hari mengajar oleh setiap lectur pada tahun ajaran 2019/2020

In [38]:
df_schedule.show()
df_course.show()

+-----------+---------+-----------+----------+----------+--------------------+
|schedule_id|course_id|lecturer_id|start_date|  end_date|         course_days|
+-----------+---------+-----------+----------+----------+--------------------+
|          1|        1|         11|2019-09-09|2019-12-08|Monday,Wednesday,...|
|          2|        3|          4|2019-09-09|2019-12-08|   Tuesday,Wednesday|
|          3|        4|         13|2019-09-09|2019-12-08|     Monday,Thursday|
|          4|        5|         28|2019-09-09|2019-12-08|             Tuesday|
|          5|        2|         56|2020-01-27|2020-04-26|    Monday,Wednesday|
|          6|        7|          1|2020-01-27|2020-04-26|  Wednesday,Thursday|
|          7|        1|         11|2020-01-27|2020-04-26|Monday,Wednesday,...|
|          8|        7|         22|2020-01-27|2020-04-26|      Monday,Tuesday|
|          9|        8|         56|2020-01-27|2020-04-26|            Thursday|
|         10|        3|         37|2020-01-27|2020-0

In [39]:
#Saya bisa menggunakan df_analys3 karena datanya sama yang dibutuhkan

df_analys4 = df_analys3
df_analys4.orderBy('start_date').show()

+-----------+-----------+--------------------+--------------------+----------+----------+
|schedule_id|lecturer_id|         course_name|         course_days|start_date|  end_date|
+-----------+-----------+--------------------+--------------------+----------+----------+
|          1|         11|Introduction to A...|Monday,Wednesday,...|2019-09-09|2019-12-08|
|          2|          4|         Data Mining|   Tuesday,Wednesday|2019-09-09|2019-12-08|
|          3|         13|Applied Data Science|     Monday,Thursday|2019-09-09|2019-12-08|
|          4|         28|           Statistic|             Tuesday|2019-09-09|2019-12-08|
|          7|         11|Introduction to A...|Monday,Wednesday,...|2020-01-27|2020-04-26|
|          5|         56|Natural Language ...|    Monday,Wednesday|2020-01-27|2020-04-26|
|         10|         37|         Data Mining|     Thursday,Friday|2020-01-27|2020-04-26|
|          8|         22|  Python Programming|      Monday,Tuesday|2020-01-27|2020-04-26|
|         

In [40]:
#Melakukan pemisahan data yang multiple pada kolom "course_days" menjadi 1-1 untuk bisa dihitung nantinya

df_analys4_split = df_analys4 \
    .withColumn('days', split(col('course_days'), ',')) \
    .withColumn('day', explode(col('days')))

df_analys4_split.show()

+-----------+-----------+--------------------+--------------------+----------+----------+--------------------+---------+
|schedule_id|lecturer_id|         course_name|         course_days|start_date|  end_date|                days|      day|
+-----------+-----------+--------------------+--------------------+----------+----------+--------------------+---------+
|          7|         11|Introduction to A...|Monday,Wednesday,...|2020-01-27|2020-04-26|[Monday, Wednesda...|   Monday|
|          7|         11|Introduction to A...|Monday,Wednesday,...|2020-01-27|2020-04-26|[Monday, Wednesda...|Wednesday|
|          7|         11|Introduction to A...|Monday,Wednesday,...|2020-01-27|2020-04-26|[Monday, Wednesda...|   Friday|
|          1|         11|Introduction to A...|Monday,Wednesday,...|2019-09-09|2019-12-08|[Monday, Wednesda...|   Monday|
|          1|         11|Introduction to A...|Monday,Wednesday,...|2019-09-09|2019-12-08|[Monday, Wednesda...|Wednesday|
|          1|         11|Introdu

In [41]:
#Melakukan analisis sesuai yang ingin dicari dengan berdasarkan data yang telah dilakukan pemisahan sebelumnya

df_lecturer_involvement = df_analys4_split \
    .groupBy('lecturer_id') \
    .count() \
    .withColumnRenamed('count', 'total_lecturing_days') \
    .orderBy('lecturer_id') \
    .withColumn('lecturer_id', col('lecturer_id').cast('string')) #Diubah formatnya dari integer menjadi string, agar pada visualisasinya menjadi rapih

df_lecturer_involvement.show()

+-----------+--------------------+
|lecturer_id|total_lecturing_days|
+-----------+--------------------+
|          1|                   2|
|          4|                   2|
|         11|                   6|
|         13|                   2|
|         22|                   2|
|         28|                   1|
|         37|                   2|
|         56|                   3|
+-----------+--------------------+



In [42]:
#Melakukan visualisasi dari data yang telah di analisis

pandas_df_lecturer_involvement = df_lecturer_involvement.toPandas()
fig4 = px.bar(pandas_df_lecturer_involvement, x='lecturer_id', y='total_lecturing_days', title= "Total Hari Mengajar Setiap Lectur Tahun Ajaran 2019/2020")
fig4.show()

### **Analisis 5 (Nilai Korelasi Analisis 1 & 2)**

Mengetahui nilai korelasi antara hasil analisis 1 dengan analisis 2.

Adapun data yang dibutuhkan adalah hasil dari analisis 1 dengan anilisis 2 yang dimana akan dilakukan join dengan berdasarkan kolom yang sama yaitu 'course_name'. Sehingga nantinya akan memperlihatkan nilai korelasinya, jika memiliki nilai mendekati 1 maka memiliki keterkaitan yang besar antara data tersebut.

In [43]:
df_analys1_clean.show()

df_analys2_clean = df_analys2.groupBy('course_name').count().withColumnRenamed('count', 'total_attendance')
df_analys2_clean.show()

+--------------------+-------------+
|         course_name|total_student|
+--------------------+-------------+
|  Python Programming|            1|
|         Data Mining|            5|
|           Statistic|            2|
|Applied Data Science|            2|
|Introduction to A...|            4|
|Natural Language ...|            1|
+--------------------+-------------+

+--------------------+----------------+
|         course_name|total_attendance|
+--------------------+----------------+
|  Python Programming|              19|
|         Data Mining|             128|
|           Statistic|              19|
|Applied Data Science|              44|
|Introduction to A...|             144|
|Natural Language ...|              26|
+--------------------+----------------+



In [44]:
#Melakukan perhitungan korelasi dengan corr antara hasil analisis 1 dengan hasil analisis 2

correlation = df_analys1_clean.join(df_analys2_clean, 'course_name', 'inner') \
    .select(corr('total_student', 'total_attendance').alias('correlation_value'))

correlation.show()

+------------------+
| correlation_value|
+------------------+
|0.9290956117547955|
+------------------+



## **4. Data Load**

Melakukan export data-data dari hasil analisis dan visualisasi.

### **Export Graph to Image**

Melakukan export dari graph yang telah dibuat menjadi image (PNG)

In [45]:
fig1.write_image("Jumlah Siswa Setiap Kursus.png")
fig2a.write_image("Total Kehadiran Siswa Tahun 2019.png")
fig2b.write_image("Total Kehadiran Siswa Tahun 2020.png")
fig3a.write_image("Total Kursus Aktif Dalam Seminggu Tahun 2019.png")
fig3b.write_image("Total Kursus Aktif Dalam Seminggu Tahun 2020.png")
fig4.write_image("Total Hari Mengajar Setiap Lectur.png")

### **Export Dataframe to CSV**

Melakukan export dataframe yang merupakan hasil analisis dan dasar data sebagai graph menjadi file CSV

In [48]:
df_analys1_clean.write.format("csv").option("header", "true").save("/content/drive/MyDrive/Day 13/Export/df_analys1")
df_analys2_filtered_2019.write.format("csv").option("header", "true").save("/content/drive/MyDrive/Day 13/Export/df_analys2_2019")
df_analys2_filtered_2020.write.format("csv").option("header", "true").save("/content/drive/MyDrive/Day 13/Export/df_analys2_2020")
df_analys3_2019_clean.write.format("csv").option("header", "true").save("/content/drive/MyDrive/Day 13/Export/df_analys3_2019")
df_analys3_2020_clean.write.format("csv").option("header", "true").save("/content/drive/MyDrive/Day 13/Export/df_analys3_2020")
df_lecturer_involvement.write.format("csv").option("header", "true").save("/content/drive/MyDrive/Day 13/Export/df_analys4")